In [1]:
import numpy as np
import xarray as xr
import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
# Years used for climatology
clim_years = [1991, 2020]

#dir = '/Volumes/Data_2TB/NMME/SST/by_lead/concatenated';
# Input/output directory
basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'
dirin = basepath+'/concatenated'#/NMME_SST/by_lead
#dirin = '/Volumes/Data_2TB/NMME/SST/by_lead/concatenated';
dirout =  basepath+'/climatology'#/NMME_SST/by_lead
#dirout = '/Volumes/Data_2TB/NMME/SST/by_lead/climatology';

# Model names
#           1              2               3            4              5             6
mods = ['CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO', 'GFDL-SPEAR', 'NASA-GEOSS2S', 'NCEP-CFSv2']#
nmod = len(mods)
nl = dict(zip(mods,[11, 11, 11, 11, 8, 9])) # Max lead time for each model
mods = [ 'NCEP-CFSv2',]#

In [3]:
# Loop through models
print('\nCalculating climatologies for NMME forecasts...\n')
for modi in mods:
    
    print(f'Processing {modi}...\n')
    
    # Loop through lead times
    for il in range(0,nl[modi]):
        
        # Load hindcast
        f_in = f'{dirin}/sst_{modi}_l{il}_concatenated_ensmean.nc'
        with xr.open_dataset(f_in,decode_times=False) as fin:
            # Find year and month (time is in months since 1960-1-1)
            time=fin.time.values
            yy=np.array([int(el/12)+1960 for el in time])
            mm=np.array([int(el%12)+1 for el in time])
            
            sst_clim=np.empty((12,len(fin.lat.values),len(fin.lon.values)))
            # Calculate climatology
            for im in range(1,13):
                ind = (mm==im) & (yy>=clim_years[0]) & (yy<=clim_years[1])
                sstsel=fin.sst[ind,:,:].data
                sst_clim[im-1,:,:]=sstsel.mean(axis=0)
                #sst_clim(:,:,im) = nanmean(sst(:,:,ind),3); 
    
            # Save to file
            f_out = f'{dirout}/sst_{modi}_l{il}_ensmean_climatology_{clim_years[0]}_{clim_years[1]}.nc'
            xout=xr.Dataset(data_vars={'lon':(['X',],fin.lon.values),
                                       'lat':(['Y',],fin.lat.values),
                                    'sst_clim':(['Mon','Y','X'],sst_clim)},
                        coords=dict(X=fin.X,Y=fin.Y,Mon=("Mon",np.arange(1,13))),)
            xout.to_netcdf(f_out,mode='w')
print('Done\n\n')


Calculating climatologies for NMME forecasts...

Processing {modi}...

Done


